# Match BGBM Collectors to Wikidata Items Using *Nearest Neighbour*

In this example we add `eventDate` of the source data, when the sample was collected, to have a time reference, when the collector was alive. 

Basically we attempt a match of `canonical_string` of WikiData to `canonical_string` of the collectors (in this case the names were parsed beforehand into single names using <https://libraries.io/rubygems/dwc_agent>)

TODO:

- review code: match when full collector name was given instead of an abbreviated collector name
- review code: evaluate if multiple names (WikiData or collector data) are found
- match also with time periode of work (WikiData) ⇌ created time of the herbarium sheet (if no other life time data are available)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os
wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", index_col=0, low_memory=False)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768.0,1826.0,NaN,NaN,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818.0,1904.0,NaN,NaN,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718.0,1790.0,NaN,NaN,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805.0,1860.0,NaN,NaN,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808.0,1880.0,NaN,NaN,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()

wd_matchtest

# colls = list(wikidata.columns)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


In [3]:
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

wd_matchtest_fullnames


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


### Load Collectors Data Set

Data sources:

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`


In [5]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("./data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
#  work around: use datetime or using pd.Periode(…)
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(lambda x: pd.Period(x, freq='ms'))

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

convert eventDate_mean to pd.Period(...) in collectors
convert eventDate_min to pd.Period(...) in collectors
convert eventDate_max to pd.Period(...) in collectors


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
1,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000
31503,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00.000,1949-03-02 00:00:00.000,1949-03-02 00:00:00.000
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000
22400,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B100217620,1997-07-10 16:00:00.000,1997-07-05 00:00:00.000,1997-07-14 00:00:00.000
22404,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B100326682,1997-07-09 12:00:00.000,1997-07-07 00:00:00.000,1997-07-12 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54129,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,5,https://herbarium.bgbm.org/object/B100263330,1988-06-17 14:24:00.000,1986-08-18 00:00:00.000,1989-09-10 00:00:00.000
27597,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100263331,1989-09-03 00:00:00.000,1989-09-03 00:00:00.000,1989-09-03 00:00:00.000
54130,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B100009590,2003-05-17 08:00:00.000,2000-08-29 00:00:00.000,2004-09-24 00:00:00.000
54131,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100530714,1951-07-18 00:00:00.000,1951-07-18 00:00:00.000,1951-07-18 00:00:00.000


#### Check Composition of Parsed Collector Data

In [6]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (622 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
1990,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B200125981,1969-11-18 00:00:00.000,1969-11-18 00:00:00.000,1969-11-18 00:00:00.000
22483,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100184021,1950-03-20 00:00:00.000,1950-03-20 00:00:00.000,1950-03-20 00:00:00.000
17179,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00.000,1938-08-24 00:00:00.000,1938-08-24 00:00:00.000
15034,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00.000,1987-06-19 00:00:00.000,1987-06-19 00:00:00.000
343,Aghababyan,Mvon,NaN,NaN,NaN,NaN,NaN,NaN,96,https://herbarium.bgbm.org/object/B100576238,2013-07-17 00:00:00.000,2013-07-15 00:00:00.000,2013-07-19 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53410,Zernig,Kurt,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B101139614,2019-06-29 08:00:00.000,2019-05-07 00:00:00.000,2019-09-28 00:00:00.000
23411,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,63,https://herbarium.bgbm.org/object/B100517202,2012-08-26 04:57:08.571,2012-08-17 00:00:00.000,2012-09-02 00:00:00.000
53444,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,3,https://je.jacq.org/JE04006629,1896-11-27 08:00:00.000,1894-04-03 00:00:00.000,1898-09-20 00:00:00.000
53445,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,1,https://je.jacq.org/JE04007139,1900-09-01 00:00:00.000,1900-09-01 00:00:00.000,1900-09-01 00:00:00.000


In [7]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    print(test_collectors.head())


----------------------------------------
show names with **particle** found 433 records:

        family      given suffix  particle  dropping_particle  nick  \
17179    Abreu  Guilherme    NaN        de                NaN   NaN   
3447   Aguilar       M.L.    NaN  Reyna de                NaN   NaN   
49438  Aguilar       M.L.    NaN  Reyna de                NaN   NaN   
13570  Aguilar       M.L.    NaN  Reyna de                NaN   NaN   
37316  Aguilar       M.L.    NaN  Reyna de                NaN   NaN   

      appellation title  occurrenceID_count  \
17179         NaN   NaN                   1   
3447          NaN   NaN                   4   
49438         NaN   NaN                  26   
13570         NaN   NaN                   2   
37316         NaN   NaN                   3   

                                occurrenceID_firstsample  \
17179  http://id.snsb.info/snsb/collection/22086/3086...   
3447        https://herbarium.bgbm.org/object/B100031063   
49438       https:/

Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [8]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)
collectors.tail()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
54129,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",5,https://herbarium.bgbm.org/object/B100263330,1988-06-17 14:24:00.000,1986-08-18 00:00:00.000,1989-09-10 00:00:00.000
27597,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",1,https://herbarium.bgbm.org/object/B100263331,1989-09-03 00:00:00.000,1989-09-03 00:00:00.000,1989-09-03 00:00:00.000
54130,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.",3,https://herbarium.bgbm.org/object/B100009590,2003-05-17 08:00:00.000,2000-08-29 00:00:00.000,2004-09-24 00:00:00.000
54131,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",1,https://herbarium.bgbm.org/object/B100530714,1951-07-18 00:00:00.000,1951-07-18 00:00:00.000,1951-07-18 00:00:00.000
54132,Жирова,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Жирова, O.",1,https://herbarium.bgbm.org/object/B100630811,1996-04-05 00:00:00.000,1996-04-05 00:00:00.000,1996-04-05 00:00:00.000


In [9]:
# group and aggregate data to have unique name rows only for the matching of names later on
# TODO 20230828 continue TypeError: Could not convert 2008-06-05 00:00:00.000 to numeric
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

collectors_unique

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.",1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00.000,1949-03-02 00:00:00.000,1949-03-02 00:00:00.000
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.",5,https://herbarium.bgbm.org/object/B100217620,1997-07-10 02:00:00.000,1997-07-05 00:00:00.000,1997-07-14 00:00:00.000
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.",1,https://herbarium.bgbm.org/object/B101153811,2002-03-24 00:00:00.000,2002-03-24 00:00:00.000,2002-03-24 00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18330,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelazny, J.",4,https://herbarium.bgbm.org/object/B100344466,2004-07-18 12:00:00.000,2004-06-09 00:00:00.000,2004-09-02 00:00:00.000
18331,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",6,https://herbarium.bgbm.org/object/B100263330,1989-01-24 19:12:00.000,1986-08-18 00:00:00.000,1989-09-10 00:00:00.000
18332,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.",3,https://herbarium.bgbm.org/object/B100009590,2003-05-17 08:00:00.000,2000-08-29 00:00:00.000,2004-09-24 00:00:00.000
18333,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",1,https://herbarium.bgbm.org/object/B100530714,1951-07-18 00:00:00.000,1951-07-18 00:00:00.000,1951-07-18 00:00:00.000


In [10]:
# show collectors with highest occurrenceID_collectors_count
collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family'], ascending=[False, True]).head(10)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
17741,Willing,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Willing, E.",189913,https://herbarium.bgbm.org/object/B100145955,1989-07-16 05:50:08.663,1919-01-01 00:00:00.000,2021-07-15 00:00:00.000
17742,Willing,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Willing, R.",188081,https://herbarium.bgbm.org/object/B100074611,2000-04-30 01:59:08.686,1976-06-15 00:00:00.000,2021-07-15 00:00:00.000
6492,Hein,P.,NaN,NaN,NaN,NaN,NaN,NaN,"Hein, P.",9340,https://herbarium.bgbm.org/object/B100113011,2001-10-31 22:48:57.640,1976-01-01 00:00:00.000,2020-09-04 00:00:00.000
8150,Kilian,N.,NaN,NaN,NaN,NaN,NaN,NaN,"Kilian, N.",9054,https://herbarium.bgbm.org/object/B100003448,2000-10-07 10:07:04.544,1982-06-01 00:00:00.000,2018-06-02 00:00:00.000
13691,Rodríguez,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Rodríguez, D.",5206,https://herbarium.bgbm.org/object/B100038970,2014-04-24 04:12:33.610,2003-05-15 00:00:00.000,2022-03-20 00:00:00.000
16219,Tempel,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Tempel, E.",4433,https://dr.jacq.org/DR073621,1978-05-30 15:21:33.661,1949-01-01 00:00:00.000,1997-03-09 00:00:00.000
6408,Haussknecht,H.K.,NaN,NaN,NaN,NaN,NaN,NaN,"Haussknecht, H.K.",4051,http://id.snsb.info/snsb/collection/474055/583...,1869-09-04 19:30:36.009,1856-01-01 00:00:00.000,1938-01-01 00:00:00.000
14487,Schimper,G.W.,NaN,NaN,NaN,NaN,NaN,NaN,"Schimper, G.W.",3806,http://id.snsb.info/snsb/collection/108223/167...,1839-04-24 02:35:51.820,1828-09-24 00:00:00.000,1938-08-19 00:00:00.000
4010,Dürbye,T.,NaN,NaN,NaN,NaN,NaN,NaN,"Dürbye, T.",3437,https://herbarium.bgbm.org/object/B100080892,2004-12-12 04:30:25.014,1905-06-15 00:00:00.000,2020-10-15 00:00:00.000
1167,Benkert,D.,NaN,NaN,NaN,NaN,NaN,NaN,"Benkert, D.",3324,https://herbarium.bgbm.org/object/B100124026,1989-07-09 01:08:50.599,1885-09-17 00:00:00.000,2006-05-07 00:00:00.000


In [11]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [12]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))

(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


In [13]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [14]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))

Show ngram examples:
- simple name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- data from collectors: [' Aa', 'Aai', 'aik', 'iki', 'ki ']
- data from match-test: [' Wa', 'Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' Oh', 'Oh ']
- data from match-test (full name): [' 18', '188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr', 'dr ']


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (BGBM) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [15]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: Number of neighbors required for each sample by default for :meth:`kneighbors` queries (originally 5).

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data with %s neighbor sample(s)...' % (len(query_data), n_neighbors))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_source_data', 'namematch_resource_data', 'namematch_distance']
    )

    print('Done')
    return matches


In [16]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
print("Calculate matching for **abbrevated** names separately …")
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

matches

Calculate abbrevated names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 2.7217233180999756 s
Getting nearest neighbours of 17858 data with 5 neighbor sample(s)...
Completed after 112.46096062660217 s
Finding matches build new data frame ...
Building matches done after 112.63547348976135 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,0,"Faber, K.","Faber, K.",0.00
1,12440,"Bertsch, K.","Bertsch, K.",0.00
2,12441,"Akeroyd, J.R.","Akeroyd, J.R.",0.00
3,4900,"Bouquet, A.","Bouquet, A.",0.00
4,4899,"Carlquist, S.","Carlquist, S.",0.00
...,...,...,...,...
17853,6218,"Vihodzevssky, N.","Smolsky, N.",1.22
17854,14283,Scabbi,"Scallan, H.",1.22
17855,17513,Kyusoz,"Nyushko, T.I.",1.22
17856,2989,"Avjetijezs, V.","Mekhtijeva, N.A.",1.23


In [17]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
print("Calculate matching for **full** names separately …")
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

matches_fullnames


Calculate matching for **full** names separately …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.7504022121429443 s
Getting nearest neighbours of 477 data with 5 neighbor sample(s)...
Completed after 10.84843397140503 s
Finding matches build new data frame ...
Building matches done after 10.852427244186401 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,317,"Körnicke, Friedrich August","Körnicke, Friedrich August",0.00
1,249,"Jaquet, Firmin","Jaquet, Firmin",0.00
2,95,"Bock, Wilhelm","Bock, Wilhelm",0.00
3,252,"Steiger, Jürg","Steiger, Jürg",0.00
4,254,"Sintenis, Paul","Sintenis, Paul",0.00
...,...,...,...,...
472,288,"Entry, Provisional","Şentürk, Ozan",1.20
473,337,"Xinjiang, Inst Bio-Ped-Psamm","Insam, J",1.21
474,426,"Rainbird, Coldecott","Colden, Jane",1.21
475,176,"Vietnitz, Oelsen-von","Schoellenbach, Carl Christoph Oelhafen von",1.22


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

In [18]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.",1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000,2008-06-05 00:00:00.000,13993,"A. Cano, E.","Cano, Á.",0.64
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00.000,1949-03-02 00:00:00.000,1949-03-02 00:00:00.000,16754,Aaiki,"Naiki, A.",0.84
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,13750,"Aaronsohn, A.","Aaronsohn, A.",0.00
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.",5,https://herbarium.bgbm.org/object/B100217620,1997-07-10 02:00:00.000,1997-07-05 00:00:00.000,1997-07-14 00:00:00.000,2548,"Abaouz, A.","Arbaoui, S.",1.12
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.",1,https://herbarium.bgbm.org/object/B101153811,2002-03-24 00:00:00.000,2002-03-24 00:00:00.000,2002-03-24 00:00:00.000,10953,"Abarca, R.","Abarca, L.",0.57


In [20]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Abdallah, Raffael",1,https://herbarium.bgbm.org/object/B200125981,1969-11-18 00:00:00.000,1969-11-18 00:00:00.000,1969-11-18 00:00:00.000,85,"Abdallah, Raffael","Issel, Raffaele",1.04
1,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,"Abdul, Kadir Bin",1,https://herbarium.bgbm.org/object/B100184021,1950-03-20 00:00:00.000,1950-03-20 00:00:00.000,1950-03-20 00:00:00.000,131,"Abdul, Kadir Bin","Kadir, Zaid bin A",1.00
2,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,"Abreu, Guilherme",1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00.000,1938-08-24 00:00:00.000,1938-08-24 00:00:00.000,422,"Abreu, Guilherme","Rau, Guilherme",0.81
3,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"Adá, García",1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00.000,1987-06-19 00:00:00.000,1987-06-19 00:00:00.000,363,"Adá, García","Adá, Ramón García",0.78
4,Aghababyan,Mvon,NaN,NaN,NaN,NaN,NaN,NaN,"Aghababyan, Mvon",96,https://herbarium.bgbm.org/object/B100576238,2013-07-17 00:00:00.000,2013-07-15 00:00:00.000,2013-07-19 00:00:00.000,271,"Aghababyan, Mvon","Aghababyan, Vladislav",0.81


In [21]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,13750,"Aaronsohn, A.","Aaronsohn, A.",0.0
7,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",2,https://herbarium.bgbm.org/object/B100241637,1946-05-22 12:00:00.000,1932-07-26 00:00:00.000,1960-03-18 00:00:00.000,5178,"Abbe, E.C.","Abbe, E.C.",0.0
10,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.",80,https://herbarium.bgbm.org/object/B100181131,1999-11-05 15:33:58.473,1992-05-20 00:00:00.000,2007-07-17 00:00:00.000,13753,"Abbott, J.R.","Abbott, J.R.",0.0
12,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",4,http://id.snsb.info/snsb/collection/504820/626...,1921-08-04 16:00:00.000,1921-04-05 00:00:00.000,1922-04-27 00:00:00.000,1401,"Abbott, W.L.","Abbott, W.L.",0.0
15,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.",13,https://herbarium.bgbm.org/object/B100046632,1971-09-20 06:00:00.000,1969-05-29 00:00:00.000,1972-05-26 00:00:00.000,6207,"Abedin, S.","Abedin, S.",0.0


In [22]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

Save the plain name matching results only ...

In [23]:
if not os.path.exists('data'):
    os.makedirs('data')

this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
    # "20230823"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_all_matches.to_csv(this_output_file)

print("Wrote plain name matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote plain name matches of collector names into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_20230828.csv (3471 kB)


### Merge and Aggregate Matched Data and WikiData’s

Review (TODO)
- evaluate time references: `eventDate` ~ `yob`, `wyb`—perhaps define a score value that could integrate all scores from properties we need for decision of the name matching (name distance, eventDate ~ year of birth/work year begin aso.)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [24]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [25]:
print("Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
print("Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …")
print("Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …")
print("      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)")
print("      … so it gets more difficult when comparing abbreviated names")
print("      … also for (source name) Kotschy, C.G.T. eventDate_max was out of scope (1900-08-01), Karl Georg Theodor Kotschy is recorded to have lived from 1813 to 1866.")

criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Abbott'))
# criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Aghababyan'))
# criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Hein,'))

collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link',
    'collectors_eventDate_min', 'collectors_eventDate_max',
    'yob', 'yod', 'wyb', 'wye'
])

Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)
Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …
Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …
      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)
      … so it gets more difficult when comparing abbreviated names
      … also for (source name) Kotschy, C.G.T. eventDate_max was out of scope (1900-08-01), Karl Georg Theodor Kotschy is recorded to have lived from 1813 to 1866.


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod,wyb,wye
9364,1,http://id.snsb.info/snsb/collection/22980/3175...,"Kotschy, C.G.","Kotschy, C.F.",0.63,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842,1839-09-30 00:00:00.000,1839-09-30 00:00:00.000,1789.0,1856.0,NaN,NaN
9365,2215,http://id.snsb.info/snsb/collection/108230/167...,"Kotschy, C.G.T.","Kotschy, C.F.",0.76,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842,1812-06-20 00:00:00.000,1900-08-01 00:00:00.000,1789.0,1856.0,NaN,NaN
9366,36,http://id.snsb.info/snsb/collection/16719/2549...,"Kotschy, K.G.T.","Kotschy, T.",0.72,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1837-01-01 00:00:00.000,1842-01-07 00:00:00.000,1813.0,1866.0,NaN,NaN
9367,294,http://id.snsb.info/snsb/collection/117808/176...,"Kotschy, T.","Kotschy, T.",0.00,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1836-01-01 00:00:00.000,1939-11-19 00:00:00.000,1813.0,1866.0,NaN,NaN
9368,3,https://herbarium.bgbm.org/object/B100526350,"Kotschy, Th","Kotschy, T.",0.50,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1834-08-01 00:00:00.000,1853-07-21 00:00:00.000,1813.0,1866.0,NaN,NaN
9369,1,https://herbarium.bgbm.org/object/B100160086,"Kotschyi, C.G.T.","Kotschy, T.",0.92,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1839-10-03 00:00:00.000,1839-10-03 00:00:00.000,1813.0,1866.0,NaN,NaN
19467,1,https://herbarium.bgbm.org/object/B101086119,"Kotschy, Carl Georg Theodor","Kotschy, Theodor",0.68,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299,1853-09-22 00:00:00.000,1853-09-22 00:00:00.000,1813.0,1866.0,NaN,NaN


Aggregate data to get atomized listings of multiple resource name matches joining by “|” aso.

In [26]:
print('Group data by canonical names (abbreviated and full name):'
      ' multiple related WD items (e.g. Q1232456), item labels, year of birth, year of death')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
    print('Run %s:   Group by wiki data’s %s, and aggregate/join item(s), labels, yob, yod '
          'by “…|…”, add new columns “…_joined” ...' % (i + 1, wd_matching_column))
    wdata_joined_items_and_others = wikidata.groupby([wd_matching_column]).agg(
        items_joined = ('item', lambda x: '|'.join(x)),
        item_labels_joined = ('itemLabel', lambda x: '|'.join(x)),
        yob_joined = ('yob', lambda x: '|'.join([str(s) for s in list(x)]) ),
        yod_joined = ('yod', lambda x: '|'.join([str(s) for s in list(x)]) )
    ).reset_index()

    # print("Done. Show examples of items having multiple matching data «|» … ")
    # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
    # wdata_joined_items[criterion].head()

    print('Run %s:   Merge all based on namematch_resource_data, add item(s) data ...' % (i + 1))
    collectors_matches_g2 = pd.merge(
        collectors_matches_g1_merged_wikidata, wdata_joined_items_and_others,
        left_on='namematch_resource_data', right_on=wd_matching_column
        , suffixes=('__wikidata_merge', '__grp_by_items')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
    collectors_matches_group = collectors_matches_g2 \
        if i == 0 \
        else pd.concat([collectors_matches_group, collectors_matches_g2], ignore_index = True)
    
print('Done')

Group data by canonical names (abbreviated and full name): multiple related WD items (e.g. Q1232456), item labels, year of birth, year of death
Run 1:   Group by wiki data’s canonical_string, and aggregate/join item(s), labels, yob, yod by “…|…”, add new columns “…_joined” ...
Run 1:   Merge all based on namematch_resource_data, add item(s) data ...
Run 1:   Build data frame “collectors_matches_group” ...
Run 2:   Group by wiki data’s canonical_string_fullname, and aggregate/join item(s), labels, yob, yod by “…|…”, add new columns “…_joined” ...
Run 2:   Merge all based on namematch_resource_data, add item(s) data ...
Run 2:   Build data frame “collectors_matches_group” ...
Done


In [27]:
print("Show examples of item_labels_joined having multiple matching data «|» … ")
criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)

collectors_matches_group[criterion].get([ # empty 
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'item_labels_joined', 'items_joined', 'yob_joined', 'yod_joined'
], default="...get: Are data empty or it has probably a wrong named column?")


Show examples of item_labels_joined having multiple matching data «|» … 


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,item_labels_joined,items_joined,yob_joined,yod_joined
8,1,https://herbarium.bgbm.org/object/B600197578,"Abbasju, M.","Abbas, A.",0.95,Alia Abbas|Abdulla Abbas,http://www.wikidata.org/entity/Q60141229|http:...,nan|nan,nan|nan
9,1,https://herbarium.bgbm.org/object/B600197578,"Abbasju, M.","Abbas, A.",0.95,Alia Abbas|Abdulla Abbas,http://www.wikidata.org/entity/Q60141229|http:...,nan|nan,nan|nan
17,4,https://herbarium.bgbm.org/object/B100005623,"Abe, L.M.","Abe, J.",0.85,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...,1946.0|nan,nan|nan
18,4,https://herbarium.bgbm.org/object/B100005623,"Abe, L.M.","Abe, J.",0.85,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...,1946.0|nan,nan|nan
19,1,https://herbarium.bgbm.org/object/B100847062,"Abeinaen, B.","Abe, J.",1.14,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...,1946.0|nan,nan|nan
...,...,...,...,...,...,...,...,...,...
19684,6,https://dr.jacq.org/DR030609,"Müller, Hermann","Müller, Hermann",0.00,Hermann Müller|Hermann Müller,http://www.wikidata.org/entity/Q68738|http://w...,1829.0|1850.0,1883.0|1927.0
19699,6,https://herbarium.bgbm.org/object/B100241696,"Palmer, Edw","Palmer, Edward",0.74,Edward Palmer|Edward Palmer,http://www.wikidata.org/entity/Q3048655|http:/...,1831.0|1821.0,1911.0|1911.0
19700,6,https://herbarium.bgbm.org/object/B100241696,"Palmer, Edw","Palmer, Edward",0.74,Edward Palmer|Edward Palmer,http://www.wikidata.org/entity/Q3048655|http:/...,1831.0|1821.0,1911.0|1911.0
19836,1,https://herbarium.bgbm.org/object/BGT0002482,"Xu, Honggui","Xu, Han",1.00,Han Xu|Han Xu,http://www.wikidata.org/entity/Q21613168|http:...,1981.0|1981.0,nan|nan


In [28]:
# check what columns we have and what we would keep for further analysis and what to drop
pprint.pprint(collectors_matches_group.columns)
# from merge: _x would means normally from left column, _y means from right column
# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'collectors_eventDate_mean', 'collectors_eventDate_min',
       'collectors_eventDate_max', 'old_index', 'namematch_source_data',
       'namematch_resource_data', 'namematch_distance', 'item', 'itemLabel',
       'surname', 'initials', 'canonical_string__wikidata_merge',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link',
       'canonical_string__grp_by_items', 'items_joined', 'item_labels_joined',
       'yob_joined', 'yod_joined', 'canonical_string',
       'canonical_string_fullname__wikidata_merge',
       'canonical_string_fullname__grp_by_items'],
      dtype='object')


Prepare data to save later on …

In [29]:
# Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
collectors_matches_group_simplified = collectors_matches_group.get(
    ['family', 'given', 'canonical_string_collector_parsed', 
      'namematch_source_data', # redundant: 'namematch_source_data' == 'canonical_string_collector_parsed'
      'namematch_resource_data', 'namematch_distance', 
      'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max', # collecors dates
      'yob_joined', 'yod_joined', # WikiData dates
      'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
      'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
    ], default="...get: Are data empty or it has probably a wrong named column?"
)
# collectors_matches_group = collectors_matches_g3
collectors_matches_group_simplified.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_group_simplified.drop_duplicates(inplace=True)
collectors_matches_group_simplified.head()

/tmp/ipykernel_8639/251392293.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.sort_values(
/tmp/ipykernel_8639/251392293.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.drop_duplicates(inplace=True)


,family,given,canonical_string_collector_parsed,namematch_source_data,namematch_resource_data,namematch_distance,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,yob_joined,...,harv,ipni,abbr,bionomia_id,items_joined,canonical_string,canonical_string_fullname,surname,initials,item_labels_joined
4,Aaronsohn,A.,"Aaronsohn, A.","Aaronsohn, A.","Aaronsohn, A.",0.0,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,1908-06-20 00:00:00.000,1876.0,...,30592,23-1,Aarons.,Q2086130,http://www.wikidata.org/entity/Q2086130,NaN,"Aaronsohn, Aaron",Aaronsohn,A.,Aaron Aaronsohn
10,Abbe,E.C.,"Abbe, E.C.","Abbe, E.C.","Abbe, E.C.",0.0,1946-05-22 12:00:00.000,1932-07-26 00:00:00.000,1960-03-18 00:00:00.000,1905.0,...,30066,26-1,Abbe,Q10274118,http://www.wikidata.org/entity/Q10274118,NaN,"Abbe, Ernst Cleveland",Abbe,E.C.,Ernst Cleveland Abbe
13,Abbott,J.R.,"Abbott, J.R.","Abbott, J.R.","Abbott, J.R.",0.0,1999-11-05 15:33:58.473,1992-05-20 00:00:00.000,2007-07-17 00:00:00.000,1968.0,...,NaN,20015671-1,J.R.Abbott,NaN,http://www.wikidata.org/entity/Q18982386,NaN,"Abbott, J Richard",Abbott,J.R.,J. Richard Abbott
15,Abbott,W.L.,"Abbott, W.L.","Abbott, W.L.","Abbott, W.L.",0.0,1921-08-04 16:00:00.000,1921-04-05 00:00:00.000,1922-04-27 00:00:00.000,1860.0,...,27518,NaN,NaN,Q635604,http://www.wikidata.org/entity/Q635604,NaN,"Abbott, William Louis",Abbott,W.L.,William Louis Abbott
21,Abedin,S.,"Abedin, S.","Abedin, S.","Abedin, S.",0.0,1971-09-20 06:00:00.000,1969-05-29 00:00:00.000,1972-05-26 00:00:00.000,1952.0,...,69097,35239-1,Abedin,NaN,http://www.wikidata.org/entity/Q16142861,NaN,"Abedin, Sultanul",Abedin,S.,Sultanul Abedin


In [30]:
# old file bgbm_collectors_matches_wikidata_items_group_concat_%s.csv
this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_%s.csv' % (
    # "20230823"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_matches_group.to_csv(this_output_file)

print("Wrote groups of collectors matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote groups of collectors matches into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_20230828.csv (11046 kB)


### Merge Data to Individual WikiData Items

For this, merge by namematch_resource_data and focus to get individual WikiData items.

In [31]:
print('Merge simply namematch_resource_data to Wiki data for abbreviated and full names... ')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):

    # join wikidata items to avh collectors matches
    #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
    #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
    # link counts of wikidata items with same canonical name string
    #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
    #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
    
    print('Run %s:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...' % (i + 1))
    collectors_matches_wd1 = pd.merge(
        collectors_all_matches, wikidata,
        left_on='namematch_resource_data', right_on=wd_matching_column,
        suffixes=('__coll_all_matches', '__wd')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
    collectors_matches_with_wdata = collectors_matches_wd1 \
        if i == 0 \
        else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)

print('Done')


Merge simply namematch_resource_data to Wiki data for abbreviated and full names... 
Run 1:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...
Run 1:   Build data frame “collectors_matches_with_wdata” ...
Run 2:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...
Run 2:   Build data frame “collectors_matches_with_wdata” ...
Done


In [32]:
pprint.pprint(collectors_matches_with_wdata.columns)
# echo "${text}" | fold --spaces | sed 's@^@#  @'

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'collectors_eventDate_mean', 'collectors_eventDate_min',
       'collectors_eventDate_max', 'old_index', 'namematch_source_data',
       'namematch_resource_data', 'namematch_distance', 'item', 'itemLabel',
       'surname', 'initials', 'canonical_string', 'canonical_string_fullname',
       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob',
       'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link',
       'ipni_link', 'bionomia_link'],
      dtype='object')


In [33]:
collectors_matches_with_wdata.drop_duplicates(inplace=True)
collectors_matches_with_wdata

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",1,...,Q2086130,1876.0,1919.0,NaN,NaN,http://www.wikidata.org/wiki/Q2086130,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23-1,https://bionomia.net/Q2086130
1,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",2,...,Q10274118,1905.0,2000.0,NaN,NaN,http://www.wikidata.org/wiki/Q10274118,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/26-1,https://bionomia.net/Q10274118
2,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.",80,...,NaN,1968.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q18982386,NaN,NaN,https://www.ipni.org/a/20015671-1,NaN
3,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",4,...,Q635604,1860.0,1936.0,NaN,NaN,http://www.wikidata.org/wiki/Q635604,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,https://bionomia.net/Q635604
4,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.",13,...,NaN,1952.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q16142861,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/35239-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19841,Entry,Provisional,NaN,NaN,NaN,NaN,NaN,NaN,"Entry, Provisional",34,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q82758799,NaN,NaN,https://www.ipni.org/a/20040762-1,NaN
19842,Rainbird,Coldecott,NaN,NaN,NaN,NaN,NaN,NaN,"Rainbird, Coldecott",1,...,Q2504068,1724.0,1766.0,NaN,NaN,http://www.wikidata.org/wiki/Q2504068,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/1730-1,https://bionomia.net/Q2504068
19843,Xinjiang,Inst Bio-Ped-Psamm,NaN,NaN,NaN,NaN,NaN,NaN,"Xinjiang, Inst Bio-Ped-Psamm",2,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36592097,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/29635-1,NaN
19844,Vietnitz,Oelsen-von,NaN,NaN,NaN,NaN,NaN,NaN,"Vietnitz, Oelsen-von",1,...,NaN,1709.0,1785.0,NaN,NaN,http://www.wikidata.org/wiki/Q21522325,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/7089-1,NaN


Save all columns for further analysis

In [34]:
# old bgbm_collectors_matches_wikidata-botanists_all-columns_%s.csv

this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv' % (
    # "20230823"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_matches_with_wdata.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_with_wdata.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items of collector matches into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_20230828.csv (8932 kB)


In [35]:
# TODO meaningful?
# remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
compact_df_tmp.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)

# old bgbm_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv
# results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv
this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_%s.csv' % (
    # "20230705"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

compact_df_tmp.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items (unique columns) of collector matches into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_20230828.csv (8608 kB)


In [36]:
# TODO further evaluation or filtering, counting, clean up aso.

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, see https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest